In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

import sys, os
import random
import numpy as np
from shutil import copy
import matplotlib.pyplot as plt
from copy import deepcopy

from omegaconf import OmegaConf
import shutil
import pickle
import random
from tqdm import tqdm
from collections import defaultdict

from torchvision.datasets.folder import ImageFolder
from torch.utils.data import DataLoader
# from skimage.filters import threshold_local, gaussian

from datetime import datetime
import pdb

In [ ]:
# run_path = "runs/178-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/179-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/180-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=2.0MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/181-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=2.0MinCont=0.1_cnext26_CUB-190-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/182-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.1MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/183-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/185-LOUSet1-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=15epsEps=60TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/191-LOUSet2-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/192-PruningNaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0NoTanhDescMinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"

# run_path = "runs/193-LOUSet3-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=20"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/195-LOUSet3-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/198-rerun-LOUSet1-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/199-rerun-LOUSet3-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_90'

# run_path = "runs/197-LOUSet3-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_90'

# run_path = "runs/200-rerun-LOUSet1-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/201-LOUSet1-PruningBF=1.1NaiveHPIPNetOODEnt=-0.2MaskL1=0.5MaskTrainExtra=05epsEps=85Cl=4.0TanhDesc=0.05MinCont=0.5_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pcOr1pd"
# ckpt_file_name = 'net_trained_90'

run_path = "runs/205-LOUSet4-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
ckpt_file_name = 'net_trained_last'

# run_path = "runs/206-LOUSet5-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/207-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_BUT-51-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/208-PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_cnext26_FISH-38-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/216-178like-cnext26_PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=1.0MinCont=0.1_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/215-178like-cnext26_PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.5MinCont=0.1_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/214-178like-cnext26_PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.1MinCont=0.1_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

# run_path = "runs/213-178like-cnext26_PruningBF=1.1NaiveHPIPNetMaskL1=0.5MaskTrainExtra=05epsEps=60Cl=2.0TanhDesc=0.05MinCont=0.1_CUB-190-imgnet-224_WeightedCE_with-equalize-aug_img=224_nprotos=10pc"
# ckpt_file_name = 'net_trained_last'

print(run_path)

In [ ]:
from pipnet.pipnet import PIPNet, get_network
from util.log import Log
from util.args import get_args, save_args, get_optimizer_nn
from util.data import get_dataloaders
from util.func import init_weights_xavier
from pipnet.train import train_pipnet, test_pipnet
# from pipnet.test import eval_pipnet, get_thresholds, eval_ood
from util.eval_cub_csv import eval_prototypes_cub_parts_csv, get_topk_cub, get_proto_patches_cub
from util.vis_pipnet import visualize, visualize_topk
from util.visualize_prediction import vis_pred, vis_pred_experiments
from util.node import Node
from util.phylo_utils import construct_phylo_tree, construct_discretized_phylo_tree
from util.func import get_patch_size
from util.data import ModifiedLabelLoader

# Save leave_out_classes to a variable

In [ ]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if ('leave_out_classes' in args) and (args.leave_out_classes != ''):
        with open(args.leave_out_classes, 'r') as file:
            leave_out_classes = [line.strip() for line in file]

# Define tree

In [ ]:
args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
args = pickle.load(args_file)

if args.phylo_config:
    phylo_config = OmegaConf.load(args.phylo_config)

if args.phylo_config:
    # construct the phylo tree
    if phylo_config.phyloDistances_string == 'None':
        if '031' in run_path: # this run uses a different phylogeny file that had an extra root node which is a mistake
            root = construct_phylo_tree('data/phlyogenyCUB/18Species-with-extra-root-node/1_tree-consensus-Hacket-18Species-modified_cub-names_v1.phy')
        else:
            root = construct_phylo_tree(phylo_config.phylogeny_path)
        print('-'*25 + ' No discretization ' + '-'*25)
    else:
        root = construct_discretized_phylo_tree(phylo_config.phylogeny_path, phylo_config.phyloDistances_string)
        print('-'*25 + ' Discretized ' + '-'*25)
else:
    # construct the tree (original hierarchy as described in the paper)
    root = Node("root")
    root.add_children(['animal','vehicle','everyday_object','weapon','scuba_diver'])
    root.add_children_to('animal',['non_primate','primate'])
    root.add_children_to('non_primate',['African_elephant','giant_panda','lion'])
    root.add_children_to('primate',['capuchin','gibbon','orangutan'])
    root.add_children_to('vehicle',['ambulance','pickup','sports_car'])
    root.add_children_to('everyday_object',['laptop','sandal','wine_bottle'])
    root.add_children_to('weapon',['assault_rifle','rifle'])
    # flat root
    # root.add_children(['scuba_diver','African_elephant','giant_panda','lion','capuchin','gibbon','orangutan','ambulance','pickup','sports_car','laptop','sandal','wine_bottle','assault_rifle','rifle'])
root.assign_all_descendents()

exp_no = int(os.path.basename(run_path)[:3])

if exp_no < 77:
    if ('num_protos_per_descendant' in args) and (args.num_protos_per_descendant > 0):
        for node in root.nodes_with_children():
            node.set_num_protos(args.num_protos_per_descendant)
if exp_no == 77:
    # update num of protos per node based on num_protos_per_descendant
    if args.num_features == 0 and args.num_protos_per_descendant == 0:
        raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
    for node in root.nodes_with_children():
        node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                                            min_protos=args.num_features)

elif 'num_protos_per_child' in args:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0 and args.num_protos_per_child == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                num_protos_per_child=args.num_protos_per_child,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))
else:
    if ('num_protos_per_descendant' in args):
        # update num of protos per node based on num_protos_per_descendant
        if args.num_features == 0 and args.num_protos_per_descendant == 0:
            raise Exception('Either of num_features or num_protos_per_descendant must be greater than zero')
        for node in root.nodes_with_children():
            node.set_num_protos(num_protos_per_descendant=args.num_protos_per_descendant,\
                                num_protos_per_child=0,\
                                min_protos=args.num_features,\
                                split_protos=('protopool' in args) and (args.protopool == 'n'))

# Load model

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    device_ids = [torch.cuda.current_device()]
else:
    device = torch.device('cpu')
    device_ids = []

# args_file = open(os.path.join(run_path, 'metadata', 'args.pickle'), 'rb')
# args = pickle.load(args_file)

# ckpt_file_name = 'net_overspecific_pruned_replaced_thresh=0.5_last'
# ckpt_file_name = 'net_trained_last'
# ckpt_file_name = 'net_trained_10'
# ckpt_file_name = 'net_pretrained'
epoch = ckpt_file_name.split('_')[-1]

ckpt_path = os.path.join(run_path, 'checkpoints', ckpt_file_name)
checkpoint = torch.load(ckpt_path, map_location=device)

if ckpt_file_name != 'net_trained_last':
    print('\n', (10*'-')+'WARNING: Not using the final trained model'+(10*'-'), '\n')

# Obtain the dataset and dataloaders
temp = args.leave_out_classes
args.leave_out_classes = '' # NOTE: because here we need to load the entire dataloader, and filter out the classes later
trainloader, trainloader_pretraining, trainloader_normal, trainloader_normal_augment, projectloader, testloader, test_projectloader, classes = get_dataloaders(args, device)
args.leave_out_classes = temp

print(args.batch_size, trainloader.batch_size)

if len(classes)<=20:
    if args.validation_size == 0.:
        print("Classes: ", testloader.dataset.class_to_idx, flush=True)
    else:
        print("Classes: ", str(classes), flush=True)

# Create a convolutional network based on arguments and add 1x1 conv layer
feature_net, add_on_layers, pool_layer, classification_layers = get_network(len(classes), args, root=root)
   
# Create a PIP-Net
net = PIPNet(num_classes=len(classes),
                    feature_net = feature_net,
                    args = args,
                    add_on_layers = add_on_layers,
                    pool_layer = pool_layer,
                    classification_layers = classification_layers,
                    num_parent_nodes = len(root.nodes_with_children()),
                    root = root
                    )
net = net.to(device=device)
net = nn.DataParallel(net, device_ids = device_ids)    
net.load_state_dict(checkpoint['model_state_dict'],strict=True)
# print(net.eval())
criterion = nn.NLLLoss(reduction='mean').to(device)

# Forward one batch through the backbone to get the latent output size
with torch.no_grad():
    xs1, _, _ = next(iter(trainloader))
    xs1 = xs1.to(device)
    _, proto_features, _, _ = net(xs1)
    wshape = proto_features['root'].shape[-1]
    args.wshape = wshape #needed for calculating image patch size
    print("Output shape: ", proto_features['root'].shape, flush=True)
    
print(args.wshape)

In [ ]:
import wandb
from util.log import Log
os.environ['WANDB_DISABLED'] = 'true'
wandb_run = wandb.init(project="pipnet", name=os.path.basename(args.log_dir), config=vars(args), reinit=False)
log = Log(args.log_dir)

In [ ]:
optimizer_net, optimizer_classifier, params_to_freeze, params_to_train, params_backbone = get_optimizer_nn(net, args)            
scheduler_net = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_net, T_max=len(trainloader)*args.epochs, eta_min=args.lr_net/100.)
if args.epochs<=30:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=5, eta_min=0.001, T_mult=1, verbose=False)
else:
    scheduler_classifier = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer_classifier, T_0=10, eta_min=0.001, T_mult=1, verbose=False)

    
if args.OOD_dataset:
    trainloader_OOD, trainloader_pretraining_OOD, trainloader_normal_OOD, trainloader_normal_augment_OOD, projectloader_OOD, testloader_OOD, test_projectloader_OOD, _ = get_dataloaders(args, device, OOD=True)
    print('-'*25 + 'Using OOD data' + '-'*25)
else:
    trainloader_OOD = trainloader_pretraining_OOD = trainloader_normal_OOD = trainloader_normal_augment_OOD = projectloader_OOD = testloader_OOD = test_projectloader_OOD = None
    print('-'*25 + 'Not using OOD data' + '-'*25)
    
if ('focal_loss' in args) and (args.focal_loss == 'y'):
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = FocalLossWrapper(device=device, alpha=1, gamma=args.focal_loss_gamma, reduction='mean').to(device)
else:
    from util.custom_losses import WeightedCrossEntropyLoss, WeightedNLLLoss, FocalLossWrapper
    criterion = WeightedNLLLoss(device=device).to(device)

# Helper functions

In [ ]:
def get_heatmap(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1])))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)


def get_heatmap_uninterpolated(latent_activation, input_image):
    image_a = latent_activation.cpu().numpy()
    image_a = (image_a - image_a.min()) / (image_a.max() - image_a.min())

    input_image = (input_image - input_image.min()) / (input_image.max() - input_image.min())
    image_b = input_image.permute(1, 2, 0).cpu().numpy()
    
    reshaped_image_a = np.array(Image.fromarray((image_a[0] * 255).astype('uint8')).resize((input_image.shape[-1], input_image.shape[-1]), \
                                                                                          resample=Image.NEAREST ))
    normalized_heatmap = (reshaped_image_a - np.min(reshaped_image_a)) / (np.max(reshaped_image_a) - np.min(reshaped_image_a))
    
    heatmap_colormap = plt.get_cmap('jet')
    heatmap_colored = heatmap_colormap(normalized_heatmap)
    
    heatmap_colored_uint8 = (heatmap_colored[:, :, :3] * 255).astype(np.uint8)
    image_a_heatmap_pillow = Image.fromarray(heatmap_colored_uint8)
    image_b_pillow = Image.fromarray((image_b * 255).astype('uint8'))
    
    result_image = Image.blend(image_b_pillow, image_a_heatmap_pillow, alpha=0.3)
    
    return np.array(result_image)

def get_bb_gaussian_threshold(latent_activation, sigma=1.0, percentile=97, extend_h=0, extend_w=0):
    # latent_activation -> []
    upscaled_similarity = get_upscaled_activation_uninterpolated(latent_activation, \
                                                                 image_size=(args.image_size, args.image_size))
    upscaled_similarity = minmaxscale(upscaled_similarity)
    upscaled_similarity = gaussian(upscaled_similarity, sigma=sigma)
    upscaled_similarity = threshold_local(upscaled_similarity, block_size=15, method='mean')
    h_min, h_max, w_min, w_max = find_top_percentile_bbox(upscaled_similarity ,percentile=97)
    h_min = max(0, h_min-extend_h)
    h_max = min(upscaled_similarity.shape[0], h_max+extend_h)
    w_min = max(0, w_min-extend_w)
    w_max = min(upscaled_similarity.shape[1], w_max+extend_w)
    return h_min, h_max, w_min, w_max


def minmaxscale(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())

from torch.utils.data import DataLoader, SequentialSampler
from torch.utils.data import DataLoader

def unshuffle_dataloader(dataloader):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=dataloader.batch_size,
        shuffle=False,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

# Fine accuracy

In [ ]:
if 'cl_weight' not in args:
    args.cl_weight = 2.0

In [ ]:
# testloader

test_info, log_dict = test_pipnet(net, testloader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=False, path_prob_softmax_tau=1)

In [ ]:
round(test_info['fine_accuracy'], 4)
test_info['fine_accuracy']

In [ ]:
torch.tensor([5, 3]).pow(1/2)

# Geometric mean accuracy

In [ ]:
leafname_to_depth = {}
leafname_to_idx = testloader.dataset.class_to_idx



def get_node_depth(node):
    depth = 0
    while node.parent:
        depth += 1
        node = node.parent
    return depth

for leaf_name in root.leaf_descendents:
    leaf_node = root.get_node(leaf_name)
    leafname_to_depth[leaf_name] = get_node_depth(leaf_node)

def get_path_prob_list(out_probs, data_idx, root, leaf_node):
    node = leaf_node
    path_prob_list = []
    while node.parent:
        parent = node.parent
        child_class_idx = parent.children_to_labels[node.name]
        path_prob_list.append(out_probs[parent.name][data_idx, child_class_idx])
        node = parent
    return path_prob_list

correct = 0
total = 0

with torch.no_grad():
    for i, (xs, ys) in tqdm(enumerate(testloader), total=len(testloader)):

        features, proto_features, pooled, out = net(xs)
        out_probs = {}
        for node_name, output_logits in out.items():
            out_probs[node_name] = torch.softmax(output_logits, dim=1)

        batch_leaf_probs_list = []
        for data_idx in range(xs.shape[0]):
            leaf_probs = [0 for _ in range(len(leafname_to_idx))]
            for leafname, class_idx in leafname_to_idx.items():
                leaf_node = root.get_node(leafname)
                path_prob_list = get_path_prob_list(out_probs, data_idx, root, leaf_node)
                depth = leafname_to_depth[leafname]
                leaf_probs[class_idx] = torch.tensor([prob**(1/depth) for prob in path_prob_list]).prod()
            batch_leaf_probs_list.append(torch.tensor(leaf_probs))
        batch_leaf_probs = torch.stack(batch_leaf_probs_list)

        correct += (batch_leaf_probs.argmax(dim=1) == ys).sum()
        total += ys.shape[0]

print('Accuracy:', correct / total)

In [ ]:
len(testloader)

# Leave one out accuracy

## Remove classes from dataloader

In [ ]:
leave_out_classes

In [ ]:
from torch.utils.data import Sampler, SubsetRandomSampler

def create_filtered_dataloader(dataloader, new_sampler, batch_size):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        sampler=new_sampler,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

leave_out_loader = testloader
# leave_out_loader = test_projectloader
unique_labels = set()
total_samples = 0
for xs, ys in testloader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("testloader Unique Labels:", unique_labels)
print("testloader total_samples:", total_samples)

# leave_out_classes = args.leave_out_classes.split(',')[2]

class_of_interest = 0
classes_to_keep = [leave_out_classes[class_of_interest]]

# classes_to_keep = [leave_out_classes[1], leave_out_classes[6], leave_out_classes[8]]
# classes_to_keep = [leave_out_classes[0], leave_out_classes[1], leave_out_classes[2], leave_out_classes[3], leave_out_classes[6], leave_out_classes[7]]


# classes_to_keep = leave_out_classes

# leave_out_classes = ['cub_093_Clark_Nutcracker']

print('classes_to_keep', classes_to_keep)
idx_of_classes_to_keep = set()
name2label = leave_out_loader.dataset.class_to_idx # param
label2name = {label:name for name, label in name2label.items()}
for label in label2name:
    # NOTE: Keeping the left out classes here
    if label2name[label] in classes_to_keep:
        idx_of_classes_to_keep.add(label)

target_indices = []
for i in range(len(leave_out_loader.dataset)):
    *_, label = leave_out_loader.dataset[i]
    if label in idx_of_classes_to_keep:
        target_indices.append(i)
sampler = SubsetRandomSampler(target_indices)
to_shuffle = False
    
leave_out_loader = create_filtered_dataloader(leave_out_loader, sampler, batch_size=1) # dataloader.batch_size
unique_labels = set()
total_samples = 0
for xs, ys in leave_out_loader:
    unique_labels.update(ys.tolist())
    total_samples += xs.shape[0]
print("leave_out_loader Unique Labels:", unique_labels)
print("leave_out_loader total_samples:", total_samples)



# print('Leave out classes', args.leave_out_classes)

In [ ]:
# leave_out_loader.dataset.class_to_idx
leave_out_classes

In [ ]:
if 'cl_weight' not in args:
    args.cl_weight = 2.0

## Check each left out image path probability

This is to see what is the probablity for an image of a left out class at each node along the path

In [ ]:
leafname_to_depth = {}
leafname_to_idx = testloader.dataset.class_to_idx

name2label = leave_out_loader.dataset.class_to_idx
label2name = {label:name for name, label in name2label.items()}

def get_node_depth(node):
    depth = 0
    while node.parent:
        depth += 1
        node = node.parent
    return depth

for leaf_name in root.leaf_descendents:
    leaf_node = root.get_node(leaf_name)
    leafname_to_depth[leaf_name] = get_node_depth(leaf_node)

def get_path_prob_list(out_probs, data_idx, root, leaf_node):
    node = leaf_node
    path_prob_list = []
    while node.parent:
        parent = node.parent
        child_class_idx = parent.children_to_labels[node.name]
        path_prob_list.append(out_probs[parent.name][data_idx, child_class_idx])
        node = parent
    return path_prob_list

def get_path_node_and_prob_list(out, out_probs, data_idx, root, leaf_node):
    node = leaf_node
    path_node_and_prob_list = []
    while node.parent:
        parent = node.parent
        child_class_idx = parent.children_to_labels[node.name]
        path_node_and_prob_list.append((node, out_probs[parent.name][data_idx, child_class_idx], \
                                        out[parent.name][data_idx, child_class_idx]))
        node = parent
    return path_node_and_prob_list

correct = 0
total = 0

with torch.no_grad():
    for i, (xs, ys) in enumerate(leave_out_loader):

        print('Ground truth:', label2name[ys.item()])

        features, proto_features, pooled, out = net(xs)
        out_probs = {}
        for node_name, output_logits in out.items():
            out_probs[node_name] = torch.softmax(output_logits, dim=1)

        batch_leaf_probs_list = []
        for data_idx in range(xs.shape[0]):
            leaf_probs = [0 for _ in range(len(leafname_to_idx))]
            leafname_to_path_node_and_prob_list = {}
            for leafname, class_idx in leafname_to_idx.items():
                leaf_node = root.get_node(leafname)
                
                path_node_and_prob_list = get_path_node_and_prob_list(out, out_probs, data_idx, root, leaf_node)
                depth = leafname_to_depth[leafname]
                leaf_probs[class_idx] = torch.tensor([prob for _, prob, _ in path_node_and_prob_list]).prod()

                leafname_to_path_node_and_prob_list[leafname] = path_node_and_prob_list
            batch_leaf_probs_list.append(torch.tensor(leaf_probs))
        batch_leaf_probs = torch.stack(batch_leaf_probs_list)

        predicted_label = np.argmax(leaf_probs)
        print('Predicted:', label2name[predicted_label])

        predicted_node = root.get_node(label2name[predicted_label])
        ground_truth_parent_node = root.get_node(label2name[ys.item()]).parent

        print('Node of certainty:', ground_truth_parent_node.name)
        print('Node of certainty in predicted path:', ground_truth_parent_node.name in [node.name for node, _, _ in leafname_to_path_node_and_prob_list[predicted_node.name]])

        print(" <- ".join([f"({node.name}|{prob:.2f}|{logit:.2f})" for node, prob, logit in leafname_to_path_node_and_prob_list[predicted_node.name]]))

        correct += (batch_leaf_probs.argmax(dim=1) == ys).sum()
        total += ys.shape[0]

        print('-'*50)

print('Accuracy:', correct / total)

## Logit distribution for descendant and non-descendant species for each node

In [ ]:
leafname_to_depth = {}
leafname_to_idx = testloader.dataset.class_to_idx
idx_to_leafname = {v: k for k, v in leafname_to_idx.items()}


def get_node_depth(node):
    depth = 0
    while node.parent:
        depth += 1
        node = node.parent
    return depth

for leaf_name in root.leaf_descendents:
    leaf_node = root.get_node(leaf_name)
    leafname_to_depth[leaf_name] = get_node_depth(leaf_node)

def get_path_prob_list(out_probs, data_idx, root, leaf_node):
    node = leaf_node
    path_prob_list = []
    while node.parent:
        parent = node.parent
        child_class_idx = parent.children_to_labels[node.name]
        path_prob_list.append(out_probs[parent.name][data_idx, child_class_idx])
        node = parent
    return path_prob_list

def findCorrespondingToMax(base, target):
    output, indices = F.max_pool2d(base, kernel_size=(26, 26), return_indices=True)# these are logits
    tensor_flattened = target.view(target.shape[0], target.shape[1], -1)
    indices_flattened = indices.view(target.shape[0], target.shape[1], -1)
    corresponding_values_in_target = torch.gather(tensor_flattened, 2, indices_flattened)
    corresponding_values_in_target = corresponding_values_in_target.view(target.shape[0],\
                                     target.shape[1], 1, 1)
    pooled_target = corresponding_values_in_target
    return pooled_target

correct = 0
total = 0

# maps node_name to list of descendant_logits and non_descendant_logits
logits = defaultdict(lambda: {'descendants': [], 'non_descendants': []})
avg_proto_activations_softmaxed = defaultdict(lambda: {'descendants': [], 'non_descendants': []})
avg_proto_activations = defaultdict(lambda: {'descendants': [], 'non_descendants': []})
with torch.no_grad():
    cl_weights_dict = {node.name: getattr(net.module, '_'+node.name+'_classification').weight for node in root.nodes_with_children()}
    for i, (xs, ys) in tqdm(enumerate(testloader), total=len(testloader)):
        features, proto_features, proto_features_softmaxed, pooled, out = net(xs, return_inner_product=True)
        pooled_ip = {}
        for node in root.nodes_with_children():
            pooled_ip[node.name] = findCorrespondingToMax(proto_features_softmaxed[node.name], proto_features[node.name])
        for data_idx in range(xs.shape[0]):
            gt_label = ys[data_idx].item()
            gt_leafname = idx_to_leafname[gt_label]
            
            for node in root.nodes_with_children():
                logit_of_prediction = out[node.name][data_idx, :].max().item()
                if gt_leafname in node.leaf_descendents:
                    logits[node.name]['descendants'].append(logit_of_prediction)
                else:
                    logits[node.name]['non_descendants'].append(logit_of_prediction)
                    
            # for node in root.nodes_with_children():
            #     predicted_child_class_idx = out[node.name][data_idx, :].argmax().item()
            #     # classification_weights = getattr(net.module, '_'+node.name+'_classification').weight
            #     classification_weights = cl_weights_dict[node.name]
            #     relevant_proto_idx = torch.nonzero(classification_weights[predicted_child_class_idx, :] > 1e-3).squeeze(-1)
            #     avg_proto_activation = pooled[node.name][data_idx, relevant_proto_idx].mean().item()
            #     if gt_leafname in node.leaf_descendents:
            #         avg_proto_activations_softmaxed[node.name]['descendants'].append(avg_proto_activation)
            #     else:
            #         avg_proto_activations_softmaxed[node.name]['non_descendants'].append(avg_proto_activation)

            #     avg_proto_activation = pooled_ip[node.name][data_idx, relevant_proto_idx].mean().item()
            #     if gt_leafname in node.leaf_descendents:
            #         avg_proto_activations[node.name]['descendants'].append(avg_proto_activation)
            #     else:
            #         avg_proto_activations[node.name]['non_descendants'].append(avg_proto_activation)

In [ ]:
# np.printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
# np.set_printoptions(precision=3)

for node in root.nodes_with_children():
    print(node.name)
    print(np.mean(logits[node.name]['descendants']).round(2), np.std(logits[node.name]['descendants']).round(2),
          np.min(logits[node.name]['descendants']).round(2), np.max(logits[node.name]['descendants']).round(2))
    if node.name != 'root':
        print(np.mean(logits[node.name]['non_descendants']).round(2), np.std(logits[node.name]['non_descendants']).round(2),
              np.min(logits[node.name]['non_descendants']).round(2), np.max(logits[node.name]['non_descendants']).round(2))
    print('-'*20)


# for node in root.nodes_with_children():
#     print(node.name)
#     print(np.mean(avg_proto_activations_softmaxed[node.name]['descendants']).round(2), np.std(avg_proto_activations_softmaxed[node.name]['descendants']).round(2),
#           np.min(avg_proto_activations_softmaxed[node.name]['descendants']).round(2), np.max(avg_proto_activations_softmaxed[node.name]['descendants']).round(2))
#     if node.name != 'root':
#         print(np.mean(avg_proto_activations_softmaxed[node.name]['non_descendants']).round(2), np.std(avg_proto_activations_softmaxed[node.name]['non_descendants']).round(2),
#               np.min(avg_proto_activations_softmaxed[node.name]['non_descendants']).round(2), np.max(avg_proto_activations_softmaxed[node.name]['non_descendants']).round(2))
#     print('-'*20)

# for node in root.nodes_with_children():
#     print(node.name)
#     print(np.mean(avg_proto_activations_softmaxed[node.name]['descendants']), \
#           np.std(avg_proto_activations_softmaxed[node.name]['descendants']))
#     if node.name != 'root':
#         print(np.mean(avg_proto_activations_softmaxed[node.name]['non_descendants']), \
#               np.std(avg_proto_activations_softmaxed[node.name]['non_descendants']))
#     print('-'*20)

# for node in root.nodes_with_children():
#     print(node.name)
#     print(np.mean(avg_proto_activations[node.name]['descendants']).round(2), \
#           np.std(avg_proto_activations[node.name]['descendants']).round(2),
#          np.min(avg_proto_activations[node.name]['descendants']).round(2),
#          np.max(avg_proto_activations[node.name]['descendants']).round(2))
#     if node.name != 'root':
#         print(np.mean(avg_proto_activations[node.name]['non_descendants']).round(2), \
#               np.std(avg_proto_activations[node.name]['non_descendants']).round(2),
#              np.min(avg_proto_activations[node.name]['non_descendants']).round(2),
#              np.max(avg_proto_activations[node.name]['non_descendants']).round(2))
#     print('-'*20)

## Plot and save histograms of desc and non-desc logits

In [ ]:
import matplotlib.pyplot as plt
import os

# data = {
#     'node1': {'descendants': [1, 2, 3], 'non_descendants': [4, 5]},
#     'node2': {'descendants': [2, 3, 4], 'non_descendants': [1, 5]}
# }

data = logits

# Directory to save images
output_dir = os.path.join(run_path, 'analysis', 'desc_and_nondesc_logit_distributions')
os.makedirs(output_dir, exist_ok=True)
bins = 50
# Plotting and saving histograms
for node_name, inner_dict in data.items():
    plt.figure()
    # Plotting the histogram for descendants
    plt.hist(inner_dict['descendants'], bins=bins, color='blue', alpha=0.5, label='Descendants', density=True)
    # Plotting the histogram for non_descendants
    plt.hist(inner_dict['non_descendants'], bins=bins, color='red', alpha=0.5, label='Non Descendants', density=True)
    plt.title(f'Distribution for {node_name}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.legend()
    # Save the figure
    plt.savefig(os.path.join(output_dir, f'{node_name}_distribution.png'))
    # plt.show()
    plt.close()

print("All histograms have been saved successfully.")


## Plot and save gaussian distribution of desc and non-desc logits

In [ ]:
import matplotlib.pyplot as plt
import os
from scipy.stats import norm


# data = {
#     'node1': {'descendants': [1, 2, 3], 'non_descendants': [4, 5]},
#     'node2': {'descendants': [2, 3, 4], 'non_descendants': [1, 5]}
# }

data = logits

# Directory to save images
output_dir = os.path.join(run_path, 'analysis', 'desc_and_nondesc_logit_distributions_gaussian')
os.makedirs(output_dir, exist_ok=True)

# Plotting and saving Gaussian curves
for node_name, inner_dict in data.items():
    plt.figure()
    # Calculate and plot for descendants
    mean_desc = np.mean(inner_dict['descendants'])
    std_desc = np.std(inner_dict['descendants'])
    xmin_desc, xmax_desc = mean_desc - 3*std_desc, mean_desc + 3*std_desc
    x_desc = np.linspace(xmin_desc, xmax_desc, 100)
    y_desc = norm.pdf(x_desc, mean_desc, std_desc)
    plt.plot(x_desc, y_desc, label='Descendants', color='blue')

    # Calculate and plot for non_descendants
    mean_non_desc = np.mean(inner_dict['non_descendants'])
    std_desc_non_desc = np.std(inner_dict['non_descendants'])
    xmin_non_desc, xmax_non_desc = mean_non_desc - 3*std_desc_non_desc, mean_non_desc + 3*std_desc_non_desc
    x_non_desc = np.linspace(xmin_non_desc, xmax_non_desc, 100)
    y_non_desc = norm.pdf(x_non_desc, mean_non_desc, std_desc_non_desc)
    plt.plot(x_non_desc, y_non_desc, label='Non Descendants', color='red')

    plt.title(f'Normal Distribution for {node_name}')
    plt.xlabel('Value')
    plt.ylabel('Probability Density')
    plt.legend()
    plt.savefig(os.path.join(output_dir, f'{node_name}_gaussian.png'))
    plt.close()

print("All Gaussian plots have been saved successfully.")

## Bayesian classifier based on logits to classify desc and non-desc images

In [ ]:
import numpy as np
from scipy.stats import norm

likelihood_nodewise = {}

def classify_point(data_A, data_B, x):
    # Calculate parameters for Distribution A
    mean_A = np.mean(data_A)
    std_A = np.std(data_A, ddof=1)  # ddof=1 for sample standard deviation
    # Calculate parameters for Distribution B
    mean_B = np.mean(data_B)
    std_B = np.std(data_B, ddof=1)
    # Calculate prior probabilities
    prior_A = 0.5  # assuming equal priors
    prior_B = 0.5
    # Calculate likelihoods using the Gaussian PDF
    likelihood_A = norm.pdf(x, mean_A, std_A)
    likelihood_B = norm.pdf(x, mean_B, std_B)
    # Calculate evidence
    evidence = likelihood_A * prior_A + likelihood_B * prior_B
    # Calculate posterior probabilities using Bayes' Theorem
    posterior_A = (likelihood_A * prior_A) / evidence
    posterior_B = (likelihood_B * prior_B) / evidence
    return posterior_A, posterior_B

# # Example usage:
# data_A = [1, 2, 3, 4, 5]  # Data for distribution A
# data_B = [6, 7, 8, 9, 10]  # Data for distribution B
# prediction_point = 5  # New data point to classify

# # Get the probabilities for the new data point
# prob_desc, prob_nondesc = classify_point(data_desc, data_nondesc, prediction_point)
# print("Probability of belonging to Distribution A:", prob_A)
# print("Probability of belonging to Distribution B:", prob_B)

total = 0
correct = 0

acc_nodewise = {node.name: {'total': 0, 'correct': 0, 'gt': [], 'pred': []} for node in root.nodes_with_children()}
# 1 -> desc
# 0 -> non-desc

with torch.no_grad():
    cl_weights_dict = {node.name: getattr(net.module, '_'+node.name+'_classification').weight for node in root.nodes_with_children()}
    for i, (xs, ys) in tqdm(enumerate(testloader), total=len(testloader)):
        features, proto_features, proto_features_softmaxed, pooled, out = net(xs, return_inner_product=True)

        for node in root.nodes_with_children():
            pooled_ip[node.name] = findCorrespondingToMax(proto_features_softmaxed[node.name], proto_features[node.name])
        for data_idx in range(xs.shape[0]):
            gt_label = ys[data_idx].item()
            gt_leafname = idx_to_leafname[gt_label]
            
            for node in root.nodes_with_children():
                logit_of_prediction = out[node.name][data_idx, :].max().item()
                prob_desc, prob_nondesc = classify_point(logits[node.name]['descendants'], \
                                                         logits[node.name]['non_descendants'], logit_of_prediction)
                if (gt_leafname in node.leaf_descendents) and (prob_desc > prob_nondesc):# and (prob_desc >= 0.55):
                    acc_nodewise[node.name]['correct'] += 1
                elif (gt_leafname not in node.leaf_descendents) and (prob_desc < prob_nondesc):# and (prob_nondesc >= 0.55):
                    acc_nodewise[node.name]['correct'] += 1
                acc_nodewise[node.name]['total'] += 1
                
                if (gt_leafname in node.leaf_descendents):
                    acc_nodewise[node.name]['gt'].append(1)
                else:
                    acc_nodewise[node.name]['gt'].append(0)

                if (prob_desc >= prob_nondesc):
                    acc_nodewise[node.name]['pred'].append(1)
                else:
                    acc_nodewise[node.name]['pred'].append(0)


print('Accuracy:', np.mean([acc['correct']/acc['total'] for _, acc in acc_nodewise.items()]))


In [ ]:
num_desc = []
acc_list = []
for node_name, acc in acc_nodewise.items():
    node = root.get_node(node_name)
    num_desc.append(len(node.leaf_descendents))
    acc_list.append(acc['correct']/acc['total'])

def plot_xy(x, y):
    plt.figure(figsize=(8, 6))  # Set the figure size
    plt.scatter(x, y, color='blue')
    # plt.plot(x, y, color='red', label='Line connecting points')
    plt.title('Plot of X vs Y')
    plt.xlabel('Accuracy')
    plt.ylabel('Num descendants')
    plt.legend()
    plt.grid(True)
    plt.show()

# Call the function with the example data
plot_xy(num_desc, acc_list)


In [ ]:
assert leave_out_loader.batch_size == 1

In [ ]:
def get_path_node_and_prob_list(out, out_probs, data_idx, root, leaf_node):
    node = leaf_node
    path_node_and_prob_list = []
    while node.parent:
        parent = node.parent
        child_class_idx = parent.children_to_labels[node.name]
        path_node_and_prob_list.append((node, out_probs[parent.name][data_idx, child_class_idx], \
                                        out[parent.name][data_idx, child_class_idx]))
        node = parent
    return path_node_and_prob_list

correct = 0
total = 0

with torch.no_grad():
    assert leave_out_loader.batch_size == 1
    for i, (xs, ys) in enumerate(leave_out_loader):
        print('Ground truth:', label2name[ys.item()])
        features, proto_features, pooled, out = net(xs)
        out_probs = {}
        for node_name, output_logits in out.items():
            out_probs[node_name] = torch.softmax(output_logits, dim=1)
        batch_leaf_probs_list = []
        desc_or_nondesc = defaultdict(list)
        for data_idx in range(xs.shape[0]):
            leaf_probs = [0 for _ in range(len(leafname_to_idx))]
            leafname_to_path_node_and_prob_list = {}
            for leafname, class_idx in leafname_to_idx.items():
                leaf_node = root.get_node(leafname)
                
                path_node_and_prob_list = get_path_node_and_prob_list(out, out_probs, data_idx, root, leaf_node)

                for node, _, logit_of_prediction in path_node_and_prob_list:
                    prob_desc, prob_nondesc = classify_point(logits[node.name]['descendants'], \
                                                         logits[node.name]['non_descendants'], logit_of_prediction.item())
                    desc_or_nondesc[leafname].append((node, prob_desc > prob_nondesc))

                depth = leafname_to_depth[leafname]
                leaf_probs[class_idx] = torch.tensor([prob for _, prob, _ in path_node_and_prob_list]).prod()

                leafname_to_path_node_and_prob_list[leafname] = path_node_and_prob_list
            batch_leaf_probs_list.append(torch.tensor(leaf_probs))
            
        batch_leaf_probs = torch.stack(batch_leaf_probs_list)
        predicted_label = np.argmax(leaf_probs)
        print('Predicted:', label2name[predicted_label])
        predicted_node = root.get_node(label2name[predicted_label])
        ground_truth_parent_node = root.get_node(label2name[ys.item()]).parent
        print('Node of certainty:', ground_truth_parent_node.name)
        print('Node of certainty in predicted path:', ground_truth_parent_node.name in [node.name for node, _, _ in leafname_to_path_node_and_prob_list[predicted_node.name]])
        # print(" <- ".join([f"({node.name}|{prob:.2f}|{logit:.2f})" for node, prob, logit in leafname_to_path_node_and_prob_list[predicted_node.name]]))
        print(" <- ".join([f"({node.name}|{d_or_nd})" for node, d_or_nd in desc_or_nondesc[predicted_node.name]]))
        correct += (batch_leaf_probs.argmax(dim=1) == ys).sum()
        total += ys.shape[0]
        print('-'*50)

print('Accuracy:', correct / total)

## Leave out without mask pruning

In [ ]:
# testloader

test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=False, leave_out_classes=leave_out_classes, path_prob_softmax_tau=1)

In [ ]:
round(test_info['fine_accuracy'], 2)
leave_out_classes

In [ ]:
# # test_info['node_accuracy'].values()

# # print(leave_out_classes[class_of_interest])

# # for key in test_info['node_accuracy']:
# #     if leave_out_classes[class_of_interest] in modified_root.get_node(key).leaf_descendents:
# #         print(key, test_info['node_accuracy'][key]['accuracy'])

# print(leave_out_classes[class_of_interest])
# leave_out_accuracy = 1
# for key in test_info['node_accuracy']:
#     if leave_out_classes[class_of_interest] in root.get_node(key).leaf_descendents:
#         if not (leave_out_classes[class_of_interest] in [child.name for child in root.get_node(key).children]):
#             leave_out_accuracy *= test_info['node_accuracy'][key]['accuracy'] / 100
#         print(key, test_info['node_accuracy'][key]['accuracy'])

# print(leave_out_accuracy)

In [ ]:
from torch.utils.data import Sampler, SubsetRandomSampler

def create_filtered_dataloader(dataloader, new_sampler, batch_size):
    if type(dataloader.dataset) == ImageFolder:
        dataset = dataloader.dataset
    else:
        dataset = dataloader.dataset.dataset.dataset
    new_dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        sampler=new_sampler,
        num_workers=dataloader.num_workers,
        pin_memory=dataloader.pin_memory,
        drop_last=dataloader.drop_last,
        timeout=dataloader.timeout,
        worker_init_fn=dataloader.worker_init_fn,
        multiprocessing_context=dataloader.multiprocessing_context,
        generator=dataloader.generator,
        prefetch_factor=dataloader.prefetch_factor,
        persistent_workers=dataloader.persistent_workers
    )
    return new_dataloader

leave_out_loader = testloader

for left_out_class in leave_out_classes:

    classes_to_keep = [left_out_class]
    
    print('classes_to_keep', classes_to_keep)
    idx_of_classes_to_keep = set()
    name2label = leave_out_loader.dataset.class_to_idx # param
    label2name = {label:name for name, label in name2label.items()}
    for label in label2name:
        # NOTE: Keeping the left out classes here
        if label2name[label] in classes_to_keep:
            idx_of_classes_to_keep.add(label)
    
    target_indices = []
    for i in range(len(leave_out_loader.dataset)):
        *_, label = leave_out_loader.dataset[i]
        if label in idx_of_classes_to_keep:
            target_indices.append(i)
    sampler = SubsetRandomSampler(target_indices)
    to_shuffle = False
        
    leave_out_loader = create_filtered_dataloader(leave_out_loader, sampler, batch_size=256) # dataloader.batch_size
    unique_labels = set()
    total_samples = 0
    for xs, ys in leave_out_loader:
        unique_labels.update(ys.tolist())
        total_samples += xs.shape[0]
    print("leave_out_loader Unique Labels:", unique_labels)
    print("leave_out_loader total_samples:", total_samples)
    
    name2label = leave_out_loader.dataset.class_to_idx
    label2name = {label:name for name, label in name2label.items()}
    
    # print('Leave out classes', args.leave_out_classes)

    # testloader

    test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                        scheduler_net, scheduler_classifier, criterion, 0, \
                                            args.epochs, device, pretrain=False, finetune=False, \
                                            test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                                tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                                minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                              args=args, apply_overspecificity_mask=False, leave_out_classes=classes_to_keep, path_prob_softmax_tau=1)
    print(left_out_class, round(test_info['fine_accuracy'], 2), '-'*10)
    print('\n')

## Leave out with mask pruning

In [ ]:
# testloader

test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
                                    scheduler_net, scheduler_classifier, criterion, 0, \
                                        args.epochs, device, pretrain=False, finetune=False, \
                                        test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
                                            tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
                                            minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
                                          args=args, apply_overspecificity_mask=True, leave_out_classes=leave_out_classes, path_prob_softmax_tau=1)

In [ ]:
# print(net.module._root_proto_presence)
# print(net.module._root_proto_presence.shape)

In [ ]:
# net.module._root_proto_presence[:, 0].shape

In [ ]:
# torch.cat([net.module._root_proto_presence[:, 1].unsqueeze(-1), net.module._root_proto_presence[:, 0].unsqueeze(-1)], dim=-1)#.shape

In [ ]:
# with torch.no_grad():
#     for node in root.nodes_with_children():
#         net.module._root_proto_presence
#         proto_presence = getattr(net.module, '_'+node.name+'_proto_presence')
#         reversed_proto_presence = torch.cat([proto_presence[:, 1].unsqueeze(-1), proto_presence[:, 0].unsqueeze(-1)], dim=-1)
#         setattr(net.module, '_'+node.name+'_proto_presence', \
#                 nn.Parameter(reversed_proto_presence))

#     test_info, log_dict = test_pipnet(net, leave_out_loader, optimizer_net, optimizer_classifier, \
#                                     scheduler_net, scheduler_classifier, criterion, 0, \
#                                         args.epochs, device, pretrain=False, finetune=False, \
#                                         test_loader_OOD=testloader_OOD, kernel_orth=args.kernel_orth == 'y', \
#                                             tanh_desc=args.tanh_desc == 'y', align=args.align == 'y', uni=args.uni == 'y', align_pf=args.align_pf == 'y',\
#                                             minmaximize=args.minmaximize == 'y', wandb_run=wandb_run, pretrain_epochs=args.epochs_pretrain, log=log, \
#                                           args=args, apply_overspecificity_mask=True)

In [ ]:
torch.tensor([False, True, True]).sum()